In [1]:
import os
from pathlib import Path
from os import path

import geopandas as gpd
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1. Input data

In [2]:
cwd = os.getcwd()
root_dir = Path(cwd).parent

In [3]:
# Input data

# Individual data
data_IND_tot = pd.read_csv(path.join(root_dir,
                                    "data/raw/individual_data.csv"))

# Household data
data_HH_tot = pd.read_csv(path.join(root_dir,
                                    "data/raw/household_data.csv"))

## Geodata (confidential, not publicly accessible)

# Mathare
mat_geo = gpd.read_file(path.join(root_dir,
                                  "data_confidential/preprocessed/data_Mat_HH_and_DiarrCases_blr.gpkg"))

# Mukuru
muk_geo = gpd.read_file(path.join(root_dir,
                                  "data_confidential/preprocessed/data_Muk_HH_and_DiarrCases_blr.gpkg"))

# 2. Preprocess data

## 2.1. Join household attributes to individuals

In [4]:
## Join tables

# Dummy dataframes
dst_data = data_IND_tot.copy().drop(['Unnamed: 0'],axis=1)
src_data = data_HH_tot.copy().drop(['Unnamed: 0','SurveyArea','Site','uuid'],axis=1)
src_data = src_data.rename(columns={"KEY": "PARENT_KEY"})

# Join
dst_data = dst_data.merge(src_data, how='left', on='PARENT_KEY')

# Check results
dst_data

,KEY,PARENT_KEY,Site,Relation_to_HH,Age,Sex,School_past,Diarrhoea,SurveyArea,HH_Number,...,HHITEMS/Microwave,HHITEMS/Oven,HHITEMS/Radio,HHITEMS/Refrigerator,HHITEMS/Smartphone,HHITEMS/Television,HHITEMS/Noitems,HHITEMS/NA,HHROOMS,StreetFood
0,uuid:2b72ea56-8a29-4874-94ae-d3f2b699f725/Rep_...,uuid:2b72ea56-8a29-4874-94ae-d3f2b699f725,Mathare,Head,25.0,M,Secondary,N,MA6,448,...,0,0,1,0,1,0,0,0,1,5_Or_More
1,uuid:335fd59e-5c49-4d12-a143-04ed378cf332/Rep_...,uuid:335fd59e-5c49-4d12-a143-04ed378cf332,Mathare,Head,38.0,F,Primary,N,MA3,711,...,0,0,1,0,1,0,0,0,1,2_4
2,uuid:335fd59e-5c49-4d12-a143-04ed378cf332/Rep_...,uuid:335fd59e-5c49-4d12-a143-04ed378cf332,Mathare,S_D,19.0,M,Secondary,N,MA3,711,...,0,0,1,0,1,0,0,0,1,2_4
3,uuid:335fd59e-5c49-4d12-a143-04ed378cf332/Rep_...,uuid:335fd59e-5c49-4d12-a143-04ed378cf332,Mathare,S_D,12.0,F,NaN,N,MA3,711,...,0,0,1,0,1,0,0,0,1,2_4
4,uuid:90692aea-e337-41c7-8d8f-0b8083b037cc/Rep_...,uuid:90692aea-e337-41c7-8d8f-0b8083b037cc,Mathare,Head,24.0,M,Primary,N,MA8,298c,...,0,0,1,0,0,0,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3781,uuid:86c835c1-c642-432c-9c69-177958aa9d29/Rep_...,uuid:86c835c1-c642-432c-9c69-177958aa9d29,Mukuru,W_H,22.0,F,Primary,N,MU3,327,...,0,0,1,0,0,1,0,0,1,5_Or_More
3782,uuid:007e2f7b-0d59-4de9-aab1-3509c6c4566f/Rep_...,uuid:007e2f7b-0d59-4de9-aab1-3509c6c4566f,Mukuru,Head,36.0,M,Secondary,N,MU6,1253,...,0,0,1,0,0,1,0,0,1,0_1
3783,uuid:007e2f7b-0d59-4de9-aab1-3509c6c4566f/Rep_...,uuid:007e2f7b-0d59-4de9-aab1-3509c6c4566f,Mukuru,W_H,28.0,F,NaN,N,MU6,1253,...,0,0,1,0,0,1,0,0,1,0_1
3784,uuid:007e2f7b-0d59-4de9-aab1-3509c6c4566f/Rep_...,uuid:007e2f7b-0d59-4de9-aab1-3509c6c4566f,Mukuru,S_D,9.0,M,NaN,N,MU6,1253,...,0,0,1,0,0,1,0,0,1,0_1


## 2.2. Set exposure variables and subsets (age groups)

In [5]:
## Set exposure variables necessary for analysis: water facilities' characteristics + potential confounders

# Dummy dataframe
df = dst_data.copy()


## Outcome of interest

# Diarrhea
diarr_mask = (df["Diarrhoea"]=='Y')
df['Case'] = np.nan
df['Case'][(~df["Diarrhoea"].isna())] = 0
df['Case'][diarr_mask] = 1


## Water services

# Recode access to "basic water" (following JMP definition) VS any other service type (including unimproved)
wt_basic_mask = (((df["DrinkingWaterGroup"]=='Private')&
                  (df["DrinkingWater_Private"].isin(['Piped_dwel','Piped_yard'])))|
                 ((df["DrinkingWaterGroup"]=='Private')&
                  (df["DrinkingWater_Private"].isin(['Piped_neigh']))&
                  (df["DrinkingWaterDist"]<=30))|
                 ((df["DrinkingWaterGroup"]=='Public')&
                  (df["DrinkingWater_Public"].isin(['Public_Tap','Vendor','Public_Dispenser','Ground_tube']))&
                  (df["DrinkingWaterDist"]<=30)))
df["BasicWt"] = np.nan
df["BasicWt"][(~df["DrinkingWaterGroup"].isna())] = 0
df["BasicWt"][wt_basic_mask] = 1

# Recode distance to water source (whether 30 min or less for a round-trip)
wt_near_mask = ((df["DrinkingWaterGroup"]=='Private')&
                (df["DrinkingWater_Private"].isin(['Piped_dwel','Piped_yard']))|
                ((df["DrinkingWaterDist"]<=30)&(~df["DrinkingWaterDist"].isna())))
df["WtSrcNear"] = np.nan
df["WtSrcNear"][(~df["DrinkingWaterGroup"].isna())] = 0
df["WtSrcNear"][wt_near_mask] = 1
    
# Recode access to water piped to premises VS any other service type (including unimproved)
wt_piped_mask = (df["DrinkingWater_Private"].isin(['Piped_dwel','Piped_yard'])) # water piped to own dwelling or yard
df["PipedWt"] = np.nan
df["PipedWt"][(~df["DrinkingWaterGroup"].isna())] = 0
df["PipedWt"][wt_piped_mask] = 1

# Recode access to piped water from neighbor VS any other service type (including unimproved)
wt_pipnb_mask = ((df["DrinkingWater_Private"].isin(['Piped_neigh']))&
                 (df["DrinkingWaterDist"]<=30)) # water piped to a neighbor's dwelling or yard
df["PipNbWt"] = np.nan
df["PipNbWt"][(~df["DrinkingWaterGroup"].isna())] = 0
df["PipNbWt"][wt_pipnb_mask] = 1

# Recode use of water from public water taps/dispensers VS any other service type (including unimproved)
wt_pubtp_mask = ((df["DrinkingWater_Public"].isin(['Public_Tap','Public_Dispenser']))&
                 (df["DrinkingWaterDist"]<=30)) # water from public tap or dispenser ('kiosk'/'ATM')
df["PbTapWt"] = np.nan
df["PbTapWt"][(~df["DrinkingWaterGroup"].isna())] = 0
df["PbTapWt"][wt_pubtp_mask] = 1

# Recode use of water from commercial source (in a public location) VS any other service type (including unimproved)
wt_cmsrc_mask = ((df["DrinkingWater_Public"]=='Vendor')&
                 (df["DrinkingWaterDist"]<=30)) # water mainly obtained from street vendors
df["CmSrcWt"] = np.nan
df["CmSrcWt"][(~df["DrinkingWaterGroup"].isna())] = 0
df["CmSrcWt"][wt_cmsrc_mask] = 1

# Recode use of water from street vendors VS any other service type (including unimproved)
wt_stven_mask = ((df["DrinkingWater_Public"]=='Vendor')&
                 (df["DrinkingWater_PublicVendor"].isin(['Vendor_Cart', 'Vendor_Bottled', 'Vendor_Sachet']))&
                 (df["DrinkingWaterDist"]<=30)) # water mainly obtained from street vendors
df["StVenWt"] = np.nan
df["StVenWt"][(~df["DrinkingWaterGroup"].isna())] = 0
df["StVenWt"][wt_stven_mask] = 1

# Recode water storage (whether household stores water they consume)
wt_strg = (df["DrinkingWaterContainer"].isna())
df["WtStored"] = np.nan
df["WtStored"][~wt_strg] = 1
df["WtStored"][wt_strg] = 0

# Recode water storage recipient (whether household uses closed recipient, when getting water from outside premises)
wt_recipok = (((df['DrinkingWaterContainer'].isin(['Plastic_Closed']))&
               (df['DrinkingWaterContainer2'].isin(['Plastic_Closed','Metal_Closed'])))|
              ((df['DrinkingWaterContainer'].isin(['Plastic_Closed']))&
               (df['DrinkingWaterContainer2'].isin(['Only_one']))))
df["WtStrgOk"] = np.nan
df["WtStrgOk"][~df["DrinkingWaterContainer"].isna()] = 0
df["WtStrgOk"][wt_recipok] = 1

# Recode water availability in the past month (self-reported)
wt_avail_mask = (df["DrinkingWaterAvailability"]=='N') # no lack of water in the previous month
df["AvailblWtM"] = np.nan
df["AvailblWtM"][~df["DrinkingWaterAvailability"].isna()] = 0
df["AvailblWtM"][wt_avail_mask] = 1


## Sanitation services

# Recode sanitation variable: "improved" (according to WHO-UNICEF's JMP) VS any other facitilty type (including unimproved and O.D.)
imprv_sn_mask = ((((df.ToiletFacilityTYPE=='DryOrCompost')&
                   (df.ToiletFacilityTYPE_Dry.isin(['Dry_ImprSlab',
                                                    'Dry_VIP',
                                                    'Dry_FreshLife'])))|
                  ((df.ToiletFacilityTYPE=='Flush')&
                   (df.ToiletFacilityTYPE_Flush.isin(['Flush_piped',
                                                      'Flush_septic_tank',
                                                      'Flush_coveredPit']))))
                 #&(df.ToiletFacilitySHARE=='N') # obs.: 'improved' sanitation does not require private facility
                )
df["ImprvSan"] = np.nan
df["ImprvSan"][(~df.ToiletFacility.isna())] = 0
df["ImprvSan"][imprv_sn_mask] = 1


## Hygiene services

# Recode "basic" hygiene (adapted, excluding 'mobile' item)
basic_hg_mask = ((df["ObsHandWashWATER"]=='Water_OK') # water available at the moment of the survey
                 &(df["ObsHandWashSOAP"]=='SoapOrDeterg') # soap (or equivalent) available at the moment of the survey
                 &(df["ObsHandWashPLACE"].isin(['Obs_Fixed'])) # presence of fixed structure to wash hands
                )
df["BasicHyg"] = np.nan
df["BasicHyg"][(~df["ObsHandWashWATER"].isna())&(~df["ObsHandWashSOAP"].isna())&(~df["ObsHandWashPLACE"].isna())] = 0
df["BasicHyg"][basic_hg_mask] = 1


## Housing conditions

# Recode overcrowding (whether number of habitants per room > 3, following 'slum' definition in SDG 11)
hab_cnt_df = df[['KEY','PARENT_KEY']].groupby('PARENT_KEY').count().reset_index() # count habitants per household
hab_cnt_df = hab_cnt_df.rename(columns={"KEY": "Cnt_Hab"})
df = df.merge(hab_cnt_df,how='left',on='PARENT_KEY') # join variable with count of houshehold members
df['HHROOMS'][df['HHROOMS'].isin([111,999])] = np.nan # exclude invalid answers & outliers regarding rooms
df['HabPerRoom'] = df['Cnt_Hab']/df['HHROOMS']
hiden_mask = (df["HabPerRoom"]>3) # high density if number of household members per room > 3
df["HiDensity"] = np.nan
df["HiDensity"][~df["HabPerRoom"].isna()] = 0
df["HiDensity"][hiden_mask] = 1


## External contamination pathways

# Recode frequency of consumption of street food
st_food_mask = (df["StreetFood"].isin(['2_4','5_Or_More'])) # twice or more per week
df["StFood"] = np.nan
df["StFood"][(~df["StreetFood"].isna())] = 0
df["StFood"][st_food_mask] = 1


## Potential confounders

# Wealth based on households assets
df["ItemsScore"]=(df["HHITEMS/Computer"]+df["HHITEMS/Electricity"]+
                  df["HHITEMS/Internet"]+df["HHITEMS/Microwave"]+
                  df["HHITEMS/Oven"]+df["HHITEMS/Radio"]+
                  df["HHITEMS/Refrigerator"]+df["HHITEMS/Smartphone"]+
                  df["HHITEMS/Television"])
df["ItemsScore"][(df["HHITEMS/Noitems"]==1)] = 0
df["ItemsScore"][(df["HHITEMS/NA"]==1)] = np.nan
wealth_mask = (df["ItemsScore"]>df["ItemsScore"].mean())# Number of items above the mean (of the surveyed households)
df["WealthyHH"] = np.nan
df["WealthyHH"][~df["ItemsScore"].isna()] = 0
df["WealthyHH"][wealth_mask] = 1

# Education level of heads of households
src = data_IND_tot[data_IND_tot.Relation_to_HH=='Head'][['PARENT_KEY',
                                                         'School_past']]# subset ed. level of HH
# Recode education level of heads of households
src['SecEduHH'] = np.nan
src['SecEduHH'][src['School_past'].isin(['No_Edu',
                                         'Early_CdE',
                                         'Primary'])] = 0 # up to primary education
src['SecEduHH'][src['School_past'].isin(['Secondary',
                                         'High_Ed'])] = 1 # secondary or higher education
src = src[~src.SecEduHH.isna()]
# In case a same household has 2 heads, merge lines
src = src[['SecEduHH','PARENT_KEY']].groupby(by="PARENT_KEY").max().reset_index()
# Attribute education level of heads of households
df = df.merge(src,on="PARENT_KEY",how='left')

In [6]:
## Subsets

# General population
dst_data = df.copy()
print("N (individuals) for general pop.:",dst_data.shape[0])

# Under 5, Abidjan
mask = (dst_data["Age"]<5)
dst_data_U5 = dst_data[mask]
print("N (individuals) for under 5:",dst_data_U5.shape[0])

N (individuals) for general pop.: 3786
N (individuals) for under 5: 491


# 3. Odds ratios from multiple logistic regressions

## 3.1. Exploratory analysis to identify candidate explanatory variables (to be included in logistic model)

### 3.1.1. Bivariate (unadjusted) odds ratio analysis to identify candidate explanatory variables for diarrhoea

In [7]:
## Set list of exposure variables

# Exploratory list
explorat_lst = [# WASH
                'WtStored', # whether water is stored in a recipient (only when obtained from a source out of premises)
                'WtStrgOk', # whether water storage recipient is adequate (closed)
                'AvailblWtM', # water available during the month preceeding the survey (self-reported)
                # Housing conditions
                'HiDensity', # whether household is overcrowded
                # External factors
                'StFood', # frequent consumption of street foods (twice or more per week)
                # Potential confounders
                'WealthyHH', # whether household has more assets than the average
                'SecEduHH', # whether head of household has secondary education
               ]

# Subsets list 1: general population
subsets_gen = [dst_data]
subsets_gen_str = ['general population']

# Subsets list 2: households with children under five years old
subsets_cU5 = [dst_data_U5]
subsets_cU5_str = ['children under five']

In [8]:
# Exclude variables with only '0's or '1's
for data in [dst_data,dst_data_U5]:
    excl_list = []
    for var in explorat_lst:
        if data[var].value_counts().shape[0]==1:
            excl_list = excl_list+[var]
if (len(excl_list)>0):
    print('the following variables will be excluded ( n = ',len(excl_list),' ):',excl_list)
    explorat_lst = list(set(explorat_lst)-set(excl_list))
else:
    print('all good')

all good


In [9]:
## Risk of diarrhoea, general population

# Calculate unadjusted odds ratios
outcome_var = "Diarrhoea"
outcome_pos = "Y"
outcome_neg = "N"
df_oddsr_gen = pd.DataFrame()
for idx, subset in enumerate(subsets_gen):
    print("------------------ ",subsets_gen_str[idx]," ------------------")
    for exposure in explorat_lst:
        # define groups
        print("Variable:",exposure)
        exposure_grp = subset[subset[exposure]==1]
        no_exposure_grp = subset[subset[exposure]==0]
        exposure_grp = exposure_grp[~exposure_grp[outcome_var].isna()]
        no_exposure_grp = no_exposure_grp[~no_exposure_grp[outcome_var].isna()]
        # set table for Fisher tests
        table = np.array([[exposure_grp[exposure_grp[outcome_var]==outcome_pos].shape[0], exposure_grp[exposure_grp[outcome_var]==outcome_neg].shape[0]],
                          [no_exposure_grp[no_exposure_grp[outcome_var]==outcome_pos].shape[0], no_exposure_grp[no_exposure_grp[outcome_var]==outcome_neg].shape[0]]])
        # calculate proportion of households with at least 1 case
        exposure_grp_prop = exposure_grp[exposure_grp[outcome_var]==outcome_pos].shape[0]/exposure_grp.shape[0]
        no_exposure_grp_prop = no_exposure_grp[no_exposure_grp[outcome_var]==outcome_pos].shape[0]/no_exposure_grp.shape[0]
        # calculate 95% CI - exposure group
        P_exp = exposure_grp_prop
        N_exp = exposure_grp.shape[0]
        CI_exp = 1.96*(np.sqrt((P_exp*(1-P_exp))/N_exp))
        # calculate 95% CI - no exposure group
        P_ne = no_exposure_grp_prop
        N_ne = no_exposure_grp.shape[0]
        CI_ne = 1.96*(np.sqrt((P_ne*(1-P_ne))/N_ne))
        # run Fisher tests for OR = 1
        oddsratio_eq1, pvalue_eq1 = stats.fisher_exact(table)
        # run Fisher tests for OR > 1
        oddsratio_greater1, pvalue_greater1 = stats.fisher_exact(table,alternative="greater")
        # run Fisher tests for OR < 1
        oddsratio_less1, pvalue_less1 = stats.fisher_exact(table,alternative="less")
        # add results to dataframe
        df_oddsr_gen = df_oddsr_gen.append([[exposure,
                                             subsets_gen_str[idx],
                                             exposure_grp_prop*100,
                                             '±'+str(round(CI_exp*100,2)),
                                             no_exposure_grp_prop*100,
                                             '±'+str(round(CI_ne*100,2)),
                                             oddsratio_eq1,
                                             pvalue_eq1,
                                             pvalue_greater1,
                                             pvalue_less1,
                                             table
                                            ]])

# Reset columns' names & index
df_oddsr_gen.columns = ['exposure variable','stratum',
                        '% outcomes, exposed','95% CI, exposed',
                        '% outcomes, non-exposed','95% CI, non-exposed',
                        'OR','p-value for OR=1',
                        'p_OR_hi_1','p_OR_lo_1',
                        'table'
                       ]
df_oddsr_gen = df_oddsr_gen.reset_index()

------------------  general population  ------------------
Variable: WtStored
Variable: WtStrgOk
Variable: AvailblWtM
Variable: HiDensity
Variable: StFood
Variable: WealthyHH
Variable: SecEduHH


In [10]:
## Risk of diarrhoea, under fives subsets

# Calculate unadjusted odds ratios
outcome_var = "Diarrhoea"
outcome_pos = "Y"
outcome_neg = "N"
df_oddsr_cU5 = pd.DataFrame()
for idx, subset in enumerate(subsets_cU5):
    print("------------------ ",subsets_cU5_str[idx]," ------------------")
    for exposure in explorat_lst:
        # define groups
        print("Variable:",exposure)
        exposure_grp = subset[subset[exposure]==1]
        no_exposure_grp = subset[subset[exposure]==0]
        exposure_grp = exposure_grp[~exposure_grp[outcome_var].isna()]
        no_exposure_grp = no_exposure_grp[~no_exposure_grp[outcome_var].isna()]
        # set table for Fisher tests
        table = np.array([[exposure_grp[exposure_grp[outcome_var]==outcome_pos].shape[0], exposure_grp[exposure_grp[outcome_var]==outcome_neg].shape[0]],
                          [no_exposure_grp[no_exposure_grp[outcome_var]==outcome_pos].shape[0], no_exposure_grp[no_exposure_grp[outcome_var]==outcome_neg].shape[0]]])
        # calculate proportion of households with at least 1 case
        exposure_grp_prop = exposure_grp[exposure_grp[outcome_var]==outcome_pos].shape[0]/exposure_grp.shape[0]
        no_exposure_grp_prop = no_exposure_grp[no_exposure_grp[outcome_var]==outcome_pos].shape[0]/no_exposure_grp.shape[0]
        # calculate 95% CI - exposure group
        P_exp = exposure_grp_prop
        N_exp = exposure_grp.shape[0]
        CI_exp = 1.96*(np.sqrt((P_exp*(1-P_exp))/N_exp))
        # calculate 95% CI - no exposure group
        P_ne = no_exposure_grp_prop
        N_ne = no_exposure_grp.shape[0]
        CI_ne = 1.96*(np.sqrt((P_ne*(1-P_ne))/N_ne))
        # run Fisher tests for OR = 1
        oddsratio_eq1, pvalue_eq1 = stats.fisher_exact(table)
        # run Fisher tests for OR > 1
        oddsratio_greater1, pvalue_greater1 = stats.fisher_exact(table,alternative="greater")
        # run Fisher tests for OR < 1
        oddsratio_less1, pvalue_less1 = stats.fisher_exact(table,alternative="less")
        # add results to dataframe
        df_oddsr_cU5 = df_oddsr_cU5.append([[exposure,
                                             subsets_cU5_str[idx],
                                             exposure_grp_prop*100,
                                             '±'+str(round(CI_exp*100,2)),
                                             no_exposure_grp_prop*100,
                                             '±'+str(round(CI_ne*100,2)),
                                             oddsratio_eq1,
                                             pvalue_eq1,
                                             pvalue_greater1,
                                             pvalue_less1,
                                             table
                                            ]])

# Reset columns' names
df_oddsr_cU5.columns = ['exposure variable','stratum',
                        '% outcomes, exposed','95% CI, exposed',
                        '% outcomes, non-exposed','95% CI, non-exposed',
                        'OR','p-value for OR=1',
                        'p_OR_hi_1','p_OR_lo_1',
                        'table'
                       ]
df_oddsr_cU5 = df_oddsr_cU5.reset_index()

------------------  children under five  ------------------
Variable: WtStored
Variable: WtStrgOk
Variable: AvailblWtM
Variable: HiDensity
Variable: StFood
Variable: WealthyHH
Variable: SecEduHH


In [11]:
## List of significant covariates

# Significance threshold (deliberately set at a looser value, for an exploratory analysis)
st = 0.1

# General population
list_gen = df_oddsr_gen['exposure variable'][df_oddsr_gen['p-value for OR=1']<st].to_list()

# Under fives subset
list_cu5 = df_oddsr_cU5['exposure variable'][df_oddsr_cU5['p-value for OR=1']<st].to_list()

# Merge lists
merged_lst = list(set(list_gen+list_cu5))

# Dataframe containing only selected covariates, by age group
df_raw_or = df_oddsr_gen[df_oddsr_gen['exposure variable'].isin(merged_lst)].append(df_oddsr_cU5[df_oddsr_cU5['exposure variable'].isin(merged_lst)]).drop(['p_OR_hi_1', 'p_OR_lo_1'],axis=1)

# Check results
print("Selected covariates (n=",len(merged_lst),"):",merged_lst)
df_raw_or

Selected covariates (n= 5 ): ['AvailblWtM', 'HiDensity', 'SecEduHH', 'WealthyHH', 'StFood']


,index,exposure variable,stratum,"% outcomes, exposed","95% CI, exposed","% outcomes, non-exposed","95% CI, non-exposed",OR,p-value for OR=1,table
2,0,AvailblWtM,general population,8.057229,±1.46,14.180025,±1.39,0.530371,1.715913e-08,"[[107, 1221], [345, 2088]]"
3,0,HiDensity,general population,12.561576,±1.44,11.201867,±1.49,1.138820,2.063128e-01,"[[255, 1775], [192, 1522]]"
4,0,StFood,general population,13.629097,±1.34,8.882784,±1.69,1.618643,4.973263e-05,"[[341, 2161], [97, 995]]"
5,0,WealthyHH,general population,11.082867,±1.39,13.009492,±1.56,0.833447,7.062572e-02,"[[218, 1749], [233, 1558]]"
6,0,SecEduHH,general population,10.907945,±1.53,12.769580,±1.56,0.836364,9.769477e-02,"[[173, 1413], [225, 1537]]"
2,0,AvailblWtM,children under five,19.021739,±5.67,30.065359,±5.14,0.546396,7.692428e-03,"[[35, 149], [92, 214]]"
3,0,HiDensity,children under five,28.762542,±5.13,20.212766,±5.74,1.593773,4.215001e-02,"[[86, 213], [38, 150]]"
4,0,StFood,children under five,28.391167,±4.96,19.078947,±6.25,1.681604,3.154570e-02,"[[90, 227], [29, 123]]"
5,0,WealthyHH,children under five,20.912548,±4.92,31.415929,±6.05,0.577262,9.473106e-03,"[[55, 208], [71, 155]]"
6,0,SecEduHH,children under five,21.848739,±5.25,32.812500,±6.64,0.572453,1.182167e-02,"[[52, 186], [63, 129]]"


### 3.1.2. Confirm selection of independent variables: test for multicollinearity with Variance Inflation Factor
- Criterion: variables with a VIF > 5 shall be discarded
- "basic" water was not included as a control variable because all exposure variables are already a type of "improved" water service according to the JMP; the control variable related to water was, hence, the time for a round-trip to the water source (whether 30 min or less)

In [12]:
# Set VIF threshold
vif_t = 5

# Set list of selected exposure + control variables
exposure_list = merged_lst.copy()
control_list = ['ImprvSan', # access to improved sanitation
                'BasicHyg', # presence of hand-washing materials AND fixed structure to wash hands (observed)
                'WtSrcNear' # whether round-trip to the water source takes 30 min or less
                ]

# Dummy dataframes: keep only observations with valid answers for all selected variables
dataset = dst_data[control_list+exposure_list].copy().dropna()

# Create table for Variance Inflation Factor (VIF)
vif_scores = pd.DataFrame() 
vif_scores["Attribute"] = control_list+exposure_list

# Calculate VIF for each feature
vif_var = "VIF Scores"
vif_scores[vif_var] = [variance_inflation_factor(dataset[control_list+exposure_list].values, i) for i in range(len(dataset[control_list+exposure_list].columns))] 

# Assess variables' VIF
if vif_scores[vif_var].max() < vif_t:
    print("All good: no significant multicollinearity")
else:
    drop_v_list = vif_scores["Attribute"][vif_scores[vif_var]>vif_t].to_list()
    print(">> WARNING: multicollinearity detected for:",
          drop_v_list)
    for var in drop_v_list:
        print(str(round(dataset[var].mean(),4)*100)+
              "% of individuals in selected households had a positive outcome for",
              '"',var,'"')

# View results
display(vif_scores)

>> WARNING: multicollinearity detected for: ['WtSrcNear']
99.95% of individuals in selected households had a positive outcome for " WtSrcNear "


,Attribute,VIF Scores
0,ImprvSan,3.076338
1,BasicHyg,1.102782
2,WtSrcNear,8.681877
3,AvailblWtM,1.539706
4,HiDensity,2.285905
5,SecEduHH,2.014786
6,WealthyHH,2.039234
7,StFood,3.103923


### 3.1.3. Confirm selection of independent variables: avoid redundancies and multicollinearity
- drop variables with VIF > 5  
- drop education of head of household ('SecEduHH'), as another socioeconomic variable is included (asset-based wealth, 'WealthyHH') and has a smaller VIF

In [13]:
# Remove redundant variables
merged_lst = list(set(vif_scores.Attribute[vif_scores['VIF Scores']<=vif_t].to_list())- # remove VIF > 5
                  set(['SecEduHH'])) # remove redundant variables

# Updated list
print("Confirmed covariates (n=",len(merged_lst),"):",merged_lst)

Confirmed covariates (n= 6 ): ['AvailblWtM', 'HiDensity', 'ImprvSan', 'WealthyHH', 'StFood', 'BasicHyg']


## 3.2. Adjusted odds of diarrhoea in the general population by water source (adjusted by sanitaton, hygiene and potentially significant variables identified in step 3.1)

In [14]:
# General population dataset
df = dst_data.copy()

# Relevant variables list (outcome, exposure and control variables)
outcome = 'Case'
exposure_list = ['BasicWt','PipedWt','PipNbWt','PbTapWt','CmSrcWt','StVenWt'] # run separate analyses, one for each exposure
control_list = merged_lst.copy()

# Create OR dataframe (initially empty)
or_wtsource_diarr_gen = pd.DataFrame()

# Discard exposure variable if all observations are exposed, or non-exposed (OR not plausible)
drop_list = []
print('==============================================================================')
print('                         CHECKING EXPOSURE VARIABLES:')
print('')
for var in exposure_list:
    if len(df[var].value_counts().values)==1:
        drop_list = drop_list+[var]
        print('>> WARNING : dropped',var,'( only 0s or 1s )')
if len(drop_list)==0:
    print('>> All good!')
elif len(drop_list)>0:
    exposure_list = sorted(list(set(exposure_list)-set(drop_list))) # update exposure variables list if needed
print('==============================================================================')

# Run logistic regressions for each source individually
for exposure in exposure_list:
    
    # Set exposure list
    if exposure in control_list:
        var_list = list(set(control_list)-set([exposure]))+[exposure]
    else:
        var_list = control_list+[exposure]
    
    # Data and logistic model
    data_reg = df[[outcome]+var_list].copy()
    data_reg = data_reg.dropna()
    logit = sm.Logit(data_reg[outcome], sm.add_constant(data_reg[var_list]))
    result = logit.fit()
    
    # Print the source type
    print('==============================================================================')
    print('                              Variable:',exposure)
    
    # Print the summary of the model
    print(result.summary())
    
    # Calculate the odds ratios and confidence intervals
    params = result.params
    conf = result.conf_int()
    signif = result.pvalues
    modelpval = result.llr_pvalue
    odds_ratios = np.exp(params)
    conf_lower = np.exp(conf[0])
    conf_upper = np.exp(conf[1])

    # Print the odds ratios and confidence intervals
    or_strat = pd.DataFrame({'OR': odds_ratios,
                             'signif': signif,
                             'Lower CI': conf_lower,
                             'Upper CI': conf_upper}).reset_index()
    or_strat['Significance'] = ''
    if modelpval < 0.05:
        or_strat['Significance'][or_strat['signif']>=0.1] = 'Not significant'
        or_strat['Significance'][or_strat['signif']<0.1] = '*'
        or_strat['Significance'][or_strat['signif']<0.05] = '**'
        or_strat['Significance'][or_strat['signif']<0.01] = '***'
        or_strat['Significance'][or_strat['signif']<0.001] = '****'
    else:
        or_strat['Significance'] = 'Not significant (LLR p-value > 5%)'
    or_strat = or_strat.drop(['signif'],axis=1)
    or_strat.columns = ['Exposure','Adjusted OR','Lower CI (95%)','Upper CI (95%)','Significance']
    or_strat = or_strat.iloc[1:]
    or_wtsource_diarr_gen = or_wtsource_diarr_gen.append(or_strat[or_strat['Exposure']==exposure],ignore_index=True)
    or_wtsource_diarr_gen = or_wtsource_diarr_gen.replace({'Exposure':{'PipedWt':'Water piped to premises',
                                                                       'PipNbWt':'Water obtained from neighbor (piped)',
                                                                       'PbTapWt':'Water obtained from public tap',
                                                                       'CmSrcWt':'Water obtained from a commercial source (vendor or boozer)',
                                                                       'StVenWt':'Water obtained from street vendor'
                                                                      }})

# View adjusted ORs for each source
or_wtsource_diarr_gen

                         CHECKING EXPOSURE VARIABLES:

>> All good!
Optimization terminated successfully.
         Current function value: 0.352275
         Iterations 6
                              Variable: BasicWt
                           Logit Regression Results                           
Dep. Variable:                   Case   No. Observations:                 2332
Model:                          Logit   Df Residuals:                     2324
Method:                           MLE   Df Model:                            7
Date:                Wed, 12 Jul 2023   Pseudo R-squ.:                 0.01718
Time:                        19:49:40   Log-Likelihood:                -821.50
converged:                       True   LL-Null:                       -835.86
Covariance Type:            nonrobust   LLR p-value:                 0.0001627
                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------

Optimization terminated successfully.
         Current function value: 0.351364
         Iterations 6
                              Variable: StVenWt
                           Logit Regression Results                           
Dep. Variable:                   Case   No. Observations:                 2332
Model:                          Logit   Df Residuals:                     2324
Method:                           MLE   Df Model:                            7
Date:                Wed, 12 Jul 2023   Pseudo R-squ.:                 0.01972
Time:                        19:49:40   Log-Likelihood:                -819.38
converged:                       True   LL-Null:                       -835.86
Covariance Type:            nonrobust   LLR p-value:                 2.681e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0565      0.188    -10.958      0.000      

,Exposure,Adjusted OR,Lower CI (95%),Upper CI (95%),Significance
0,BasicWt,0.962213,0.273391,3.386556,Not significant
1,Water piped to premises,0.923811,0.560660,1.522183,Not significant
2,Water obtained from neighbor (piped),0.879134,0.613452,1.259881,Not significant
3,Water obtained from public tap,0.996167,0.746993,1.328456,Not significant
4,Water obtained from a commercial source (vendo...,2.030046,1.026747,4.013732,**
5,Water obtained from street vendor,2.732996,1.137529,6.566224,**


In [15]:
# Set VIF threshold
vif_t = 5

for var in exposure_list:
    print("------------------------ ",var," ------------------------")
    # Dummy dataframes: keep only observations with valid answers for all selected variables
    dataset = dst_data[control_list+[var]].copy().dropna()

    # Create table for Variance Inflation Factor (VIF)
    vif_scores = pd.DataFrame() 
    vif_scores["Attribute"] = control_list+[var]

    # Calculate VIF for each feature
    vif_var = "VIF Scores"
    vif_scores[vif_var] = [variance_inflation_factor(dataset[control_list+[var]].values, i) for i in range(len(dataset[control_list+[var]].columns))] 

    # Assess variables' VIF
    if vif_scores[vif_var].max() < vif_t:
        print("All good: no significant multicollinearity")
    else:
        drop_v_list = vif_scores["Attribute"][vif_scores[vif_var]>vif_t].to_list()
        print(">> WARNING: multicollinearity detected for:",
              drop_v_list)
        for var in drop_v_list:
            print(str(round(dataset[var].mean(),4)*100)+
                  "% of individuals in selected households had a positive outcome for",
                  '"',var,'"')

    # View results
    display(vif_scores)

------------------------  BasicWt  ------------------------
>> WARNING: multicollinearity detected for: ['BasicWt']
99.22999999999999% of individuals in selected households had a positive outcome for " BasicWt "


,Attribute,VIF Scores
0,AvailblWtM,1.599671
1,HiDensity,2.167795
2,ImprvSan,3.111746
3,WealthyHH,2.008268
4,StFood,3.026741
5,BasicHyg,1.102939
6,BasicWt,7.529634


------------------------  PipedWt  ------------------------
All good: no significant multicollinearity


,Attribute,VIF Scores
0,AvailblWtM,1.435977
1,HiDensity,1.992633
2,ImprvSan,2.330793
3,WealthyHH,1.831912
4,StFood,2.210143
5,BasicHyg,1.095151
6,PipedWt,1.115950


------------------------  PipNbWt  ------------------------
All good: no significant multicollinearity


...

,Attribute,VIF Scores
0,AvailblWtM,1.515881
1,HiDensity,2.136401
2,ImprvSan,2.495374
3,WealthyHH,1.831643
4,StFood,2.396963
5,BasicHyg,1.088406
6,PbTapWt,2.933265


------------------------  CmSrcWt  ------------------------
All good: no significant multicollinearity


,Attribute,VIF Scores
0,AvailblWtM,1.438123
1,HiDensity,1.992019
2,ImprvSan,2.307970
3,WealthyHH,1.828328
4,StFood,2.213888
5,BasicHyg,1.089339
6,CmSrcWt,1.044538


------------------------  StVenWt  ------------------------
All good: no significant multicollinearity


,Attribute,VIF Scores
0,AvailblWtM,1.456392
1,HiDensity,1.992459
2,ImprvSan,2.330870
3,WealthyHH,1.809739
4,StFood,2.220805
5,BasicHyg,1.087593
6,StVenWt,1.042448


## 3.3. Adjusted odds of diarrhoea in children under five by water source (adjusted by sanitaton, hygiene and potentially significant variables identified in step 3.1)

In [16]:
# Children under five dataset
df = dst_data_U5.copy()

# Relevant variables list (outcome, exposure and control variables)
outcome = 'Case'
exposure_list = ['BasicWt','PipedWt','PipNbWt','PbTapWt','CmSrcWt','StVenWt'] # run separate analyses, one for each exposure
control_list = merged_lst.copy()

# Create OR dataframe (initially empty)
or_wtsource_diarr_u5 = pd.DataFrame()

# Discard exposure variable if all observations are exposed, or non-exposed (OR not plausible)
drop_list = []
print('==============================================================================')
print('                         CHECKING EXPOSURE VARIABLES:')
print('')
for var in exposure_list:
    if len(df[var].value_counts().values)==1:
        drop_list = drop_list+[var]
        print('>> WARNING : dropped',var,'( only 0s or 1s )')
if len(drop_list)==0:
    print('>> All good!')
elif len(drop_list)>0:
    exposure_list = sorted(list(set(exposure_list)-set(drop_list))) # update exposure variables list if needed
print('==============================================================================')

# Run logistic regressions for each source individually
for exposure in exposure_list:
    
    # Set exposure list
    if exposure in control_list:
        var_list = list(set(control_list)-set([exposure]))+[exposure]
    else:
        var_list = control_list+[exposure]
    
    # Data and logistic model
    data_reg = df[[outcome]+var_list].copy()
    data_reg = data_reg.dropna()
    logit = sm.Logit(data_reg[outcome], sm.add_constant(data_reg[var_list]))
    result = logit.fit()
    
    # Print the source type
    print('==============================================================================')
    print('                              Variable:',exposure)
    
    # Print the summary of the model
    print(result.summary())
    
    # Calculate the odds ratios and confidence intervals
    params = result.params
    conf = result.conf_int()
    signif = result.pvalues
    modelpval = result.llr_pvalue
    odds_ratios = np.exp(params)
    conf_lower = np.exp(conf[0])
    conf_upper = np.exp(conf[1])

    # Print the odds ratios and confidence intervals
    or_strat = pd.DataFrame({'OR': odds_ratios,
                             'signif': signif,
                             'Lower CI': conf_lower,
                             'Upper CI': conf_upper}).reset_index()
    or_strat['Significance'] = ''
    if modelpval < 0.05:
        or_strat['Significance'][or_strat['signif']>=0.1] = 'Not significant'
        or_strat['Significance'][or_strat['signif']<0.1] = '*'
        or_strat['Significance'][or_strat['signif']<0.05] = '**'
        or_strat['Significance'][or_strat['signif']<0.01] = '***'
        or_strat['Significance'][or_strat['signif']<0.001] = '****'
    else:
        or_strat['Significance'] = 'Not significant (LLR p-value > 5%)'
    or_strat = or_strat.drop(['signif'],axis=1)
    or_strat.columns = ['Exposure','Adjusted OR','Lower CI (95%)','Upper CI (95%)','Significance']
    or_strat = or_strat.iloc[1:]
    or_wtsource_diarr_u5 = or_wtsource_diarr_u5.append(or_strat[or_strat['Exposure']==exposure],ignore_index=True)
    or_wtsource_diarr_u5 = or_wtsource_diarr_u5.replace({'Exposure':{'PipedWt':'Water piped to premises',
                                                                     'PipNbWt':'Water obtained from neighbor (piped)',
                                                                     'PbTapWt':'Water obtained from public tap',
                                                                     'CmSrcWt':'Water obtained from a commercial source (vendor or boozer)',
                                                                     'StVenWt':'Water obtained from street vendor'
                                                                    }})

# View adjusted ORs for each source
or_wtsource_diarr_u5

                         CHECKING EXPOSURE VARIABLES:

>> All good!
         Current function value: 0.565237
         Iterations: 35
                              Variable: BasicWt
                           Logit Regression Results                           
Dep. Variable:                   Case   No. Observations:                  305
Model:                          Logit   Df Residuals:                      297
Method:                           MLE   Df Model:                            7
Date:                Wed, 12 Jul 2023   Pseudo R-squ.:                 0.03964
Time:                        19:49:40   Log-Likelihood:                -172.40
converged:                      False   LL-Null:                       -179.51
Covariance Type:            nonrobust   LLR p-value:                   0.04723
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -13.2392    467

Optimization terminated successfully.
         Current function value: 0.566362
         Iterations 5
                              Variable: StVenWt
                           Logit Regression Results                           
Dep. Variable:                   Case   No. Observations:                  305
Model:                          Logit   Df Residuals:                      297
Method:                           MLE   Df Model:                            7
Date:                Wed, 12 Jul 2023   Pseudo R-squ.:                 0.03772
Time:                        19:49:40   Log-Likelihood:                -172.74
converged:                       True   LL-Null:                       -179.51
Covariance Type:            nonrobust   LLR p-value:                   0.05991
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.2927      0.384     -3.370      0.001      

,Exposure,Adjusted OR,Lower CI (95%),Upper CI (95%),Significance
0,BasicWt,159417.217635,0.000000,inf,Not significant
1,Water piped to premises,1.027234,0.352517,2.993360,Not significant (LLR p-value > 5%)
2,Water obtained from neighbor (piped),1.876738,0.974726,3.613471,*
3,Water obtained from public tap,0.561043,0.319121,0.986366,**
4,Water obtained from a commercial source (vendo...,2.403296,0.653050,8.844397,Not significant
5,Water obtained from street vendor,1.775390,0.314842,10.011400,Not significant (LLR p-value > 5%)


In [17]:
# Set VIF threshold
vif_t = 5

for var in exposure_list:
    print("------------------------ ",var," ------------------------")
    # Dummy dataframes: keep only observations with valid answers for all selected variables
    dataset = dst_data_U5[control_list+[var]].copy().dropna()

    # Create table for Variance Inflation Factor (VIF)
    vif_scores = pd.DataFrame() 
    vif_scores["Attribute"] = control_list+[var]

    # Calculate VIF for each feature
    vif_var = "VIF Scores"
    vif_scores[vif_var] = [variance_inflation_factor(dataset[control_list+[var]].values, i) for i in range(len(dataset[control_list+[var]].columns))] 

    # Assess variables' VIF
    if vif_scores[vif_var].max() < vif_t:
        print("All good: no significant multicollinearity")
    else:
        drop_v_list = vif_scores["Attribute"][vif_scores[vif_var]>vif_t].to_list()
        print(">> WARNING: multicollinearity detected for:",
              drop_v_list)
        for var in drop_v_list:
            print(str(round(dataset[var].mean(),4)*100)+
                  "% of individuals in selected households had a positive outcome for",
                  '"',var,'"')
            
    # View results
    display(vif_scores)

------------------------  BasicWt  ------------------------
>> WARNING: multicollinearity detected for: ['BasicWt']
99.67% of individuals in selected households had a positive outcome for " BasicWt "


,Attribute,VIF Scores
0,AvailblWtM,1.691036
1,HiDensity,2.584612
2,ImprvSan,2.988028
3,WealthyHH,2.028344
4,StFood,2.980196
5,BasicHyg,1.074122
6,BasicWt,7.767867


------------------------  PipedWt  ------------------------
All good: no significant multicollinearity


,Attribute,VIF Scores
0,AvailblWtM,1.536806
1,HiDensity,2.307517
2,ImprvSan,2.340918
3,WealthyHH,1.843755
4,StFood,2.258916
5,BasicHyg,1.121286
6,PipedWt,1.231058


------------------------  PipNbWt  ------------------------
All good: no significant multicollinearity


...

,Attribute,VIF Scores
0,AvailblWtM,1.613540
1,HiDensity,2.472219
2,ImprvSan,2.418262
3,WealthyHH,1.776392
4,StFood,2.452815
5,BasicHyg,1.067752
6,PbTapWt,2.884520


------------------------  CmSrcWt  ------------------------
All good: no significant multicollinearity


,Attribute,VIF Scores
0,AvailblWtM,1.550242
1,HiDensity,2.316348
2,ImprvSan,2.320341
3,WealthyHH,1.772511
4,StFood,2.205947
5,BasicHyg,1.069216
6,CmSrcWt,1.061830


------------------------  StVenWt  ------------------------
All good: no significant multicollinearity


,Attribute,VIF Scores
0,AvailblWtM,1.603752
1,HiDensity,2.315506
2,ImprvSan,2.323639
3,WealthyHH,1.752655
4,StFood,2.211839
5,BasicHyg,1.067775
6,StVenWt,1.073988


# 4. Supplementary analyses

## 4.1. Associations between lack of water and diarrhea

In [18]:
# General population dataset
df = dst_data.copy()

# Relevant variables list (outcome, exposure and control variables)
outcome = 'Case'
exposure_list = ['AvailblWtM']
control_list = ['ImprvSan', 'BasicHyg', 'WealthyHH', 'HiDensity', 'StFood']

# Create OR dataframe (initially empty)
or_wtavar_diarr_gen = pd.DataFrame()

# Discard exposure variable if all observations are exposed, or non-exposed (OR not plausible)
drop_list = []
print('==============================================================================')
print('                         CHECKING EXPOSURE VARIABLES:')
print('')
for var in exposure_list:
    if len(df[var].value_counts().values)==1:
        drop_list = drop_list+[var]
        print('>> WARNING : dropped',var,'( only 0s or 1s )')
if len(drop_list)==0:
    print('>> All good!')
elif len(drop_list)>0:
    exposure_list = sorted(list(set(exposure_list)-set(drop_list))) # update exposure variables list if needed
print('==============================================================================')

# Run logistic regressions for each source individually
for exposure in exposure_list:
    
    # Set exposure list
    if exposure in control_list:
        var_list = list(set(control_list)-set([exposure]))+[exposure]
    else:
        var_list = control_list+[exposure]
    
    # Data and logistic model
    data_reg = df[[outcome]+var_list].copy()
    data_reg = data_reg.dropna()
    logit = sm.Logit(data_reg[outcome], sm.add_constant(data_reg[var_list]))
    result = logit.fit()
    
    # Print the source type
    print('==============================================================================')
    print('                              Variable:',exposure)
    
    # Print the summary of the model
    print(result.summary())
    
    # Calculate the odds ratios and confidence intervals
    params = result.params
    conf = result.conf_int()
    signif = result.pvalues
    modelpval = result.llr_pvalue
    odds_ratios = np.exp(params)
    conf_lower = np.exp(conf[0])
    conf_upper = np.exp(conf[1])

    # Print the odds ratios and confidence intervals
    or_strat = pd.DataFrame({'OR': odds_ratios,
                             'signif': signif,
                             'Lower CI': conf_lower,
                             'Upper CI': conf_upper}).reset_index()
    or_strat['Significance'] = ''
    if modelpval < 0.05:
        or_strat['Significance'][or_strat['signif']>=0.1] = 'Not significant'
        or_strat['Significance'][or_strat['signif']<0.1] = '*'
        or_strat['Significance'][or_strat['signif']<0.05] = '**'
        or_strat['Significance'][or_strat['signif']<0.01] = '***'
        or_strat['Significance'][or_strat['signif']<0.001] = '****'
    else:
        or_strat['Significance'] = 'Not significant (LLR p-value > 5%)'
    or_strat = or_strat.drop(['signif'],axis=1)
    or_strat.columns = ['Exposure','Adjusted OR','Lower CI (95%)','Upper CI (95%)','Significance']
    or_strat = or_strat.iloc[1:]
    or_wtavar_diarr_gen = or_wtavar_diarr_gen.append(or_strat[or_strat['Exposure']==exposure],ignore_index=True)
    or_wtavar_diarr_gen = or_wtavar_diarr_gen.replace({'Exposure':{'PipedWt':'Water piped to premises',
                                                           'PbTapWt':'Water obtained from public tap',
                                                           'StVenWt':'Water obtained from street vendor',
                                                          }})

# View adjusted ORs for each source
or_wtavar_diarr_gen

                         CHECKING EXPOSURE VARIABLES:

>> All good!
Optimization terminated successfully.
         Current function value: 0.352161
         Iterations 6
                              Variable: AvailblWtM
                           Logit Regression Results                           
Dep. Variable:                   Case   No. Observations:                 2333
Model:                          Logit   Df Residuals:                     2326
Method:                           MLE   Df Model:                            6
Date:                Wed, 12 Jul 2023   Pseudo R-squ.:                 0.01722
Time:                        19:49:40   Log-Likelihood:                -821.59
converged:                       True   LL-Null:                       -835.99
Covariance Type:            nonrobust   LLR p-value:                 6.660e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------

,Exposure,Adjusted OR,Lower CI (95%),Upper CI (95%),Significance
0,AvailblWtM,0.589125,0.440108,0.7886,****


In [19]:
# Set VIF threshold
vif_t = 5

# Dummy dataframes: keep only observations with valid answers for all selected variables
dataset = dst_data[control_list+exposure_list].copy().dropna()

# Create table for Variance Inflation Factor (VIF)
vif_scores = pd.DataFrame() 
vif_scores["Attribute"] = control_list+exposure_list

# Calculate VIF for each feature
vif_var = "VIF Scores"
vif_scores[vif_var] = [variance_inflation_factor(dataset[control_list+exposure_list].values, i) for i in range(len(dataset[control_list+exposure_list].columns))] 

# Assess variables' VIF
if vif_scores[vif_var].max() < vif_t:
    print("All good: no significant multicollinearity")
else:
    drop_v_list = vif_scores["Attribute"][vif_scores[vif_var]>vif_t].to_list()
    print(">> WARNING: multicollinearity detected for:",
          drop_v_list)    
    for var in drop_v_list:
        print(str(round(dataset[var].mean(),4)*100)+
              "% of individuals in selected households had a positive outcome for",
              '"',var,'"')

# View results
display(vif_scores)

All good: no significant multicollinearity


,Attribute,VIF Scores
0,ImprvSan,2.308388
1,BasicHyg,1.086421
2,WealthyHH,1.801704
3,HiDensity,1.990715
4,StFood,2.210199
5,AvailblWtM,1.435152


In [20]:
# Children under five dataset
df = dst_data_U5.copy()

# Relevant variables list (outcome, exposure and control variables)
outcome = 'Case'
exposure_list = ['AvailblWtM']
control_list = ['ImprvSan', 'BasicHyg', 'WealthyHH', 'HiDensity', 'StFood']

# Create OR dataframe (initially empty)
or_wtavar_diarr_u5 = pd.DataFrame()

# Discard exposure variable if all observations are exposed, or non-exposed (OR not plausible)
drop_list = []
print('==============================================================================')
print('                         CHECKING EXPOSURE VARIABLES:')
print('')
for var in exposure_list:
    if len(df[var].value_counts().values)==1:
        drop_list = drop_list+[var]
        print('>> WARNING : dropped',var,'( only 0s or 1s )')
if len(drop_list)==0:
    print('>> All good!')
elif len(drop_list)>0:
    exposure_list = sorted(list(set(exposure_list)-set(drop_list))) # update exposure variables list if needed
print('==============================================================================')

# Run logistic regressions for each source individually
for exposure in exposure_list:
    
    # Set exposure list
    if exposure in control_list:
        var_list = list(set(control_list)-set([exposure]))+[exposure]
    else:
        var_list = control_list+[exposure]
    
    # Data and logistic model
    data_reg = df[[outcome]+var_list].copy()
    data_reg = data_reg.dropna()
    logit = sm.Logit(data_reg[outcome], sm.add_constant(data_reg[var_list]))
    result = logit.fit()
    
    # Print the source type
    print('==============================================================================')
    print('                              Variable:',exposure)
    
    # Print the summary of the model
    print(result.summary())
    
    # Calculate the odds ratios and confidence intervals
    params = result.params
    conf = result.conf_int()
    signif = result.pvalues
    modelpval = result.llr_pvalue
    odds_ratios = np.exp(params)
    conf_lower = np.exp(conf[0])
    conf_upper = np.exp(conf[1])

    # Print the odds ratios and confidence intervals
    or_strat = pd.DataFrame({'OR': odds_ratios,
                             'signif': signif,
                             'Lower CI': conf_lower,
                             'Upper CI': conf_upper}).reset_index()
    or_strat['Significance'] = ''
    if modelpval < 0.05:
        or_strat['Significance'][or_strat['signif']>=0.1] = 'Not significant'
        or_strat['Significance'][or_strat['signif']<0.1] = '*'
        or_strat['Significance'][or_strat['signif']<0.05] = '**'
        or_strat['Significance'][or_strat['signif']<0.01] = '***'
        or_strat['Significance'][or_strat['signif']<0.001] = '****'
    else:
        or_strat['Significance'] = 'Not significant (LLR p-value > 5%)'
    or_strat = or_strat.drop(['signif'],axis=1)
    or_strat.columns = ['Exposure','Adjusted OR','Lower CI (95%)','Upper CI (95%)','Significance']
    or_strat = or_strat.iloc[1:]
    or_wtavar_diarr_u5 = or_wtavar_diarr_u5.append(or_strat[or_strat['Exposure']==exposure],ignore_index=True)
    or_wtavar_diarr_u5 = or_wtavar_diarr_u5.replace({'Exposure':{'PipedWt':'Water piped to premises',
                                                           'PbTapWt':'Water obtained from public tap',
                                                           'StVenWt':'Water obtained from street vendor',
                                                          }})

# View adjusted ORs for each source
or_wtavar_diarr_u5

                         CHECKING EXPOSURE VARIABLES:

>> All good!
Optimization terminated successfully.
         Current function value: 0.567007
         Iterations 5
                              Variable: AvailblWtM
                           Logit Regression Results                           
Dep. Variable:                   Case   No. Observations:                  305
Model:                          Logit   Df Residuals:                      298
Method:                           MLE   Df Model:                            6
Date:                Wed, 12 Jul 2023   Pseudo R-squ.:                 0.03663
Time:                        19:49:40   Log-Likelihood:                -172.94
converged:                       True   LL-Null:                       -179.51
Covariance Type:            nonrobust   LLR p-value:                   0.04071
                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------

,Exposure,Adjusted OR,Lower CI (95%),Upper CI (95%),Significance
0,AvailblWtM,0.720729,0.4209,1.234139,Not significant


In [21]:
# Set VIF threshold
vif_t = 5

# Dummy dataframes: keep only observations with valid answers for all selected variables
dataset = dst_data_U5[control_list+exposure_list].copy().dropna()

# Create table for Variance Inflation Factor (VIF)
vif_scores = pd.DataFrame() 
vif_scores["Attribute"] = control_list+exposure_list

# Calculate VIF for each feature
vif_var = "VIF Scores"
vif_scores[vif_var] = [variance_inflation_factor(dataset[control_list+exposure_list].values, i) for i in range(len(dataset[control_list+exposure_list].columns))] 

# Assess variables' VIF
if vif_scores[vif_var].max() < vif_t:
    print("All good: no significant multicollinearity")
else:
    drop_v_list = vif_scores["Attribute"][vif_scores[vif_var]>vif_t].to_list()
    print(">> WARNING: multicollinearity detected for:",
          drop_v_list)
    for var in drop_v_list:
        print(str(round(dataset[var].mean(),4)*100)+
              "% of individuals in selected households had a positive outcome for",
              '"',var,'"')

# View results
display(vif_scores)

All good: no significant multicollinearity


,Attribute,VIF Scores
0,ImprvSan,2.318292
1,BasicHyg,1.067699
2,WealthyHH,1.749490
3,HiDensity,2.302563
4,StFood,2.202059
5,AvailblWtM,1.533430


## 4.2. Descriptive statistics

### 4.2.1. All households
Basic demographic statistics for the population living in participating households

In [22]:
## All valid household surveys

# Dummy dataframe
df = dst_data.copy()

In [23]:
## Number of individuals

# Both sites
tot_ind = df.shape[0]
tot_hh = len(df.PARENT_KEY.unique())

# Mathare
tot_ind_mat = df[df['Site']=='Mathare'].shape[0]
tot_hh_mat = len(df[df['Site']=='Mathare'].PARENT_KEY.unique())

# Mukuru
tot_ind_muk = df[df['Site']=='Mukuru'].shape[0]
tot_hh_muk = len(df[df['Site']=='Mukuru'].PARENT_KEY.unique())

# Print stats
print('Total individuals:',tot_ind)
print('Total households:',tot_hh)
print('----------------------------------------')
print('Total individuals in Mathare:',tot_ind_mat)
print('Total households in Mathare:',tot_hh_mat)
print('----------------------------------------')
print('Total individuals in Mukuru:',tot_ind_muk)
print('Total households in Mukuru:',tot_hh_muk)

Total individuals: 3786
Total households: 1147
----------------------------------------
Total individuals in Mathare: 1935
Total households in Mathare: 576
----------------------------------------
Total individuals in Mukuru: 1851
Total households in Mukuru: 571


In [24]:
## Number of individuals, by gender

# Both sites
tot_ind_m = df[df['Sex']=='M'].shape[0]
tot_ind_f = df[df['Sex']=='F'].shape[0]

# Mathare
tot_ind_mat_m = df[(df['Sex']=='M')&(df['Site']=='Mathare')].shape[0]
tot_ind_mat_f = df[(df['Sex']=='F')&(df['Site']=='Mathare')].shape[0]

# Mukuru
tot_ind_muk_m = df[(df['Sex']=='M')&(df['Site']=='Mukuru')].shape[0]
tot_ind_muk_f = df[(df['Sex']=='F')&(df['Site']=='Mukuru')].shape[0]

# Print stats
print('Total male individuals:',tot_ind_m,
      '(',round((tot_ind_m/tot_ind)*100,2),'% )')
print('Total female individuals:',tot_ind_f,
      '(',round((tot_ind_f/tot_ind)*100,2),'% )')
print('----------------------------------------')
print('Total male individuals in Mathare:',tot_ind_mat_m,
      '(',round((tot_ind_mat_m/tot_ind_mat)*100,2),'% )')
print('Total female individuals in Mathare:',tot_ind_mat_f,
      '(',round((tot_ind_mat_f/tot_ind_mat)*100,2),'% )')
print('----------------------------------------')
print('Total male individuals in Mukuru:',tot_ind_muk_m,
      '(',round((tot_ind_muk_m/tot_ind_muk)*100,2),'% )')
print('Total female individuals in Mukuru:',tot_ind_muk_f,
      '(',round((tot_ind_muk_f/tot_ind_muk)*100,2),'% )')

Total male individuals: 1775 ( 46.88 % )
Total female individuals: 2011 ( 53.12 % )
----------------------------------------
Total male individuals in Mathare: 854 ( 44.13 % )
Total female individuals in Mathare: 1081 ( 55.87 % )
----------------------------------------
Total male individuals in Mukuru: 921 ( 49.76 % )
Total female individuals in Mukuru: 930 ( 50.24 % )


In [25]:
## Number of individuals, by age

# Both sites
tot_ind_ag_u5 = df[df['Age']<5].shape[0]
tot_ind_ag_ad = df[(df['Age']>18)&(df['Age']<=90)].shape[0]

# Azito
tot_ind_mat_ag_u5 = df[(df['Age']<5)&(df['Site']=='Mathare')].shape[0]
tot_ind_mat_ag_ad = df[(df['Age']>18)&(df['Age']<=90)&(df['Site']=='Mathare')].shape[0]

# Mukuru
tot_ind_muk_ag_u5 = df[(df['Age']<5)&(df['Site']=='Mukuru')].shape[0]
tot_ind_muk_ag_ad = df[(df['Age']>18)&(df['Age']<=90)&(df['Site']=='Mukuru')].shape[0]

# Print stats
print('Total individuals under five:',tot_ind_ag_u5,
      '(',round((tot_ind_ag_u5/tot_ind)*100,2),'% )')
print('Total adult individuals:',tot_ind_ag_ad,
      '(',round((tot_ind_ag_ad/tot_ind)*100,2),'% )')
print('----------------------------------------')
print('Total individuals under five in Mathare:',tot_ind_mat_ag_u5,
      '(',round((tot_ind_mat_ag_u5/tot_ind_mat)*100,2),'% )')
print('Total adult individuals in Mathare:',tot_ind_mat_ag_ad,
      '(',round((tot_ind_mat_ag_u5/tot_ind_mat)*100,2),'% )')
print('----------------------------------------')
print('Total individuals under five in Mukuru:',tot_ind_muk_ag_u5,
      '(',round((tot_ind_muk_ag_u5/tot_ind_muk)*100,2),'% )')
print('Total adult individuals in Mukuru:',tot_ind_muk_ag_ad,
      '(',round((tot_ind_muk_ag_u5/tot_ind_muk)*100,2),'% )')


Total individuals under five: 491 ( 12.97 % )
Total adult individuals: 2159 ( 57.03 % )
----------------------------------------
Total individuals under five in Mathare: 226 ( 11.68 % )
Total adult individuals in Mathare: 1091 ( 11.68 % )
----------------------------------------
Total individuals under five in Mukuru: 265 ( 14.32 % )
Total adult individuals in Mukuru: 1068 ( 14.32 % )


### 4.2.2. Households included in the models
Basic demographic statistics for the population living in participating households with valid answers for all variables included in the multiple logistic regressions

In [26]:
## Households with valid answers for all 7 variables included in the regressions

# Dummy dataframe
df = dst_data.copy()

# Subset data
df = df[['PARENT_KEY','Site','Case']+exposure_list+merged_lst+['Sex','Age']]
df = df.dropna()

In [27]:
## Number of individuals

# Both sites
tot_ind = df.shape[0]
tot_hh = len(df.PARENT_KEY.unique())

# Mathare
tot_ind_mat = df[df['Site']=='Mathare'].shape[0]
tot_hh_mat = len(df[df['Site']=='Mathare'].PARENT_KEY.unique())

# Mukuru
tot_ind_muk = df[df['Site']=='Mukuru'].shape[0]
tot_hh_muk = len(df[df['Site']=='Mukuru'].PARENT_KEY.unique())

# Print stats
print('Total individuals (MLRs):',tot_ind)
print('Total households (MLRs):',tot_hh)
print('----------------------------------------')
print('Total individuals in Mathare (MLRs):',tot_ind_mat)
print('Total households in Mathare (MLRs):',tot_hh_mat)
print('----------------------------------------')
print('Total individuals in Mukuru (MLRs):',tot_ind_muk)
print('Total households in Mukuru (MLRs):',tot_hh_muk)

Total individuals (MLRs): 2333
Total households (MLRs): 704
----------------------------------------
Total individuals in Mathare (MLRs): 1194
Total households in Mathare (MLRs): 353
----------------------------------------
Total individuals in Mukuru (MLRs): 1139
Total households in Mukuru (MLRs): 351


#### Notes
- A significant number of households (386 out of 1147) did not allow the field enumerator to observe their hygiene facilities in situ. This observation was necessary to construct the "basic hygiene" variable, which is why the number of housholds/individuals included in the multiple logistic regressions is smaller (n=703/n=2332) than the total (n=1147/n=3786).  
- Of relevance, this reduction did not affect the overall proportion of males/females or that of children younger than five years old, as shown below.

In [28]:
## Number of individuals, by gender

# Both sites
tot_ind_m = df[df['Sex']=='M'].shape[0]
tot_ind_f = df[df['Sex']=='F'].shape[0]

# Mathare
tot_ind_mat_m = df[(df['Sex']=='M')&(df['Site']=='Mathare')].shape[0]
tot_ind_mat_f = df[(df['Sex']=='F')&(df['Site']=='Mathare')].shape[0]

# Mukuru
tot_ind_muk_m = df[(df['Sex']=='M')&(df['Site']=='Mukuru')].shape[0]
tot_ind_muk_f = df[(df['Sex']=='F')&(df['Site']=='Mukuru')].shape[0]

# Print stats
print('Total male individuals (MLRs):',tot_ind_m,
      '(',round((tot_ind_m/tot_ind)*100,2),'% )')
print('Total female individuals (MLRs):',tot_ind_f,
      '(',round((tot_ind_f/tot_ind)*100,2),'% )')
print('----------------------------------------')
print('Total male individuals in Mathare (MLRs):',tot_ind_mat_m,
      '(',round((tot_ind_mat_m/tot_ind_mat)*100,2),'% )')
print('Total female individuals in Mathare (MLRs):',tot_ind_mat_f,
      '(',round((tot_ind_mat_f/tot_ind_mat)*100,2),'% )')
print('----------------------------------------')
print('Total male individuals in Mukuru (MLRs):',tot_ind_muk_m,
      '(',round((tot_ind_muk_m/tot_ind_muk)*100,2),'% )')
print('Total female individuals in Mukuru (MLRs):',tot_ind_muk_f,
      '(',round((tot_ind_muk_f/tot_ind_muk)*100,2),'% )')

Total male individuals (MLRs): 1096 ( 46.98 % )
Total female individuals (MLRs): 1237 ( 53.02 % )
----------------------------------------
Total male individuals in Mathare (MLRs): 518 ( 43.38 % )
Total female individuals in Mathare (MLRs): 676 ( 56.62 % )
----------------------------------------
Total male individuals in Mukuru (MLRs): 578 ( 50.75 % )
Total female individuals in Mukuru (MLRs): 561 ( 49.25 % )


In [29]:
## Number of individuals, by age

# Both sites
tot_ind_ag_u5 = df[df['Age']<5].shape[0]
tot_ind_ag_ad = df[(df['Age']>18)&(df['Age']<=90)].shape[0]

# Azito
tot_ind_mat_ag_u5 = df[(df['Age']<5)&(df['Site']=='Mathare')].shape[0]
tot_ind_mat_ag_ad = df[(df['Age']>18)&(df['Age']<=90)&(df['Site']=='Mathare')].shape[0]

# Mukuru
tot_ind_muk_ag_u5 = df[(df['Age']<5)&(df['Site']=='Mukuru')].shape[0]
tot_ind_muk_ag_ad = df[(df['Age']>18)&(df['Age']<=90)&(df['Site']=='Mukuru')].shape[0]

# Print stats
print('Total individuals under five (MLRs):',tot_ind_ag_u5,
      '(',round((tot_ind_ag_u5/tot_ind)*100,2),'% )')
print('Total adult individuals (MLRs):',tot_ind_ag_ad,
      '(',round((tot_ind_ag_ad/tot_ind)*100,2),'% )')
print('----------------------------------------')
print('Total individuals under five in Mathare (MLRs):',tot_ind_mat_ag_u5,
      '(',round((tot_ind_mat_ag_u5/tot_ind_mat)*100,2),'% )')
print('Total adult individuals in Mathare (MLRs):',tot_ind_mat_ag_ad,
      '(',round((tot_ind_mat_ag_u5/tot_ind_mat)*100,2),'% )')
print('----------------------------------------')
print('Total individuals under five in Mukuru (MLRs):',tot_ind_muk_ag_u5,
      '(',round((tot_ind_muk_ag_u5/tot_ind_muk)*100,2),'% )')
print('Total adult individuals in Mukuru (MLRs):',tot_ind_muk_ag_ad,
      '(',round((tot_ind_muk_ag_u5/tot_ind_muk)*100,2),'% )')


Total individuals under five (MLRs): 305 ( 13.07 % )
Total adult individuals (MLRs): 1336 ( 57.27 % )
----------------------------------------
Total individuals under five in Mathare (MLRs): 149 ( 12.48 % )
Total adult individuals in Mathare (MLRs): 664 ( 12.48 % )
----------------------------------------
Total individuals under five in Mukuru (MLRs): 156 ( 13.7 % )
Total adult individuals in Mukuru (MLRs): 672 ( 13.7 % )
